<a href="https://colab.research.google.com/github/Bhadra-Indranil/LLM-for-Mental-Health/blob/main/GPT2_QA_Mental_Health_UL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df1_2020 = pd.read_csv("20200325_counsel_chat_processed.csv")
df1_2020.head()

,Unnamed: 0,questionTitle,questionText,topic,answerText,clean_answerText,clean_questionTitle,clean_questionText
0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"If everyone thinks you're worthless, then mayb...","if everyone thinks you're worthless, then mayb...",can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
1,1,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...","hello, and thank you for your question and see...",can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
2,2,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,First thing I'd suggest is getting the sleep y...,first thing i'd suggest is getting the sleep y...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
3,3,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,Therapy is essential for those that are feelin...,therapy is essential for those that are feelin...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
4,4,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,I first want to let you know that you are not ...,i first want to let you know that you are not ...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...


In [3]:
df1_2020 = df1_2020.drop(columns=['questionTitle','questionText','topic','answerText','clean_questionTitle'])
df1 = df1_2020.dropna()
df1.head()

,Unnamed: 0,clean_answerText,clean_questionText
0,0,"if everyone thinks you're worthless, then mayb...",i'm going through some things with my feelings...
1,1,"hello, and thank you for your question and see...",i'm going through some things with my feelings...
2,2,first thing i'd suggest is getting the sleep y...,i'm going through some things with my feelings...
3,3,therapy is essential for those that are feelin...,i'm going through some things with my feelings...
4,4,i first want to let you know that you are not ...,i'm going through some things with my feelings...


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel,Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

In [5]:
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        # Create item dictionary with tensors for each key.
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        # Set labels equal to input_ids for language modeling.
        item["labels"] = item["input_ids"].clone()
        return item

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
train_data = df1.sample(frac=0.8, random_state=42)
test_data = df1.drop(train_data.index)

In [8]:
train_data['prompt'] = "Question: " + train_data['clean_questionText'] + " Answer: " + train_data['clean_answerText']
texts = train_data['prompt'].tolist()

In [9]:
model.config.pad_token_id = tokenizer.eos_token_id

In [11]:
dataset = SimpleDataset(texts, tokenizer, max_length=128)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_steps=500,
    do_train=True
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
#
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: indranil9053rkm (indranil123iitd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
50,3.454800
100,3.042000
150,2.894200
200,2.843400
250,2.806900
300,2.662400
350,2.712500
400,2.606000
450,2.594700
500,2.445500


TrainOutput(global_step=4260, training_loss=1.8486823724469108, metrics={'train_runtime': 905.4977, 'train_samples_per_second': 18.807, 'train_steps_per_second': 4.705, 'total_flos': 1112450826240000.0, 'train_loss': 1.8486823724469108, 'epoch': 10.0})

In [13]:
trainer.save_model("./fine_tuned_gpt2_in_mental_health_question")

In [14]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2_in_mental_health_question")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set pad token to EOS and update the model config
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Determine the device (GPU if available, else CPU) and move the model to it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define your test prompt (using the same format as during training)
prompts = [
    "Question: I've been feeling anxious about the future and it's affecting my daily routine. What are some effective coping strategies? Answer:",
    "Question: I've been experiencing persistent feelings of depression and hopelessness. What steps can I take to improve my mood and seek help? Answer:",
    "Question: I often feel overwhelmed by stress at work and in my personal life. What techniques can help me manage stress better? Answer:",
    "Question: I struggle with negative self-talk and low self-esteem. How can I build a more positive self-image? Answer:",
    "Question: I'm finding it hard to cope with sudden changes and uncertainty in my life. What practices can help me build resilience? Answer:",
    "Question: I feel isolated and disconnected from those around me. What can I do to create a stronger support network? Answer:",
    "Question: I have trouble sleeping due to anxiety and racing thoughts. What are some effective methods to improve my sleep quality? Answer:",
    "Question: I occasionally experience panic attacks that leave me feeling out of control. What strategies might help me prevent or manage these episodes? Answer:"
]

for prompt in prompts:
    # Tokenize the prompt and move the resulting tensor to the same device as the model
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)  # Ensure attention to all tokens

    # Generate text from the prompt
    sample_outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Ensure attention mask is passed
        max_length=1024,  # Allow longer responses
        num_beams=5,  # Beam search for better coherence
        no_repeat_ngram_size=2,  # Reduce repetition
        temperature=0.7,  # Balanced creativity
        top_p=0.9,  # Prevent extreme randomness
        eos_token_id=tokenizer.eos_token_id,  # Ensure stopping
    )

    # Decode and print the generated text
    for i, sample_output in enumerate(sample_outputs):
        text = tokenizer.decode(sample_output, skip_special_tokens=True)
        print(f"Generated Output {i+1}:\n{text}\n")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I've been feeling anxious about the future and it's affecting my daily routine. What are some effective coping strategies? Answer: anxiety is a normal response to changes in one or more areas of your life. it is not unusual to experience anxiety in the short term (i.e., a decrease in appetite) or long-term (e.g., increased risk of heart disease, stroke, or kidney failure). however, anxiety can become so severe that it can lead to major depressive episodes (mystomies) and suicidal thoughts (mentally, physically, and emotionally). i would encourage you to seek out treatment options that meet your needs.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I've been experiencing persistent feelings of depression and hopelessness. What steps can I take to improve my mood and seek help? Answer: the first step is to recognize that depression is not something to be ashamed of. depression can be a sign of a deeper problem that needs addressing. i would encourage you to speak with a mental health professional about the root of the problem and how it may be contributing to your depression.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated Output 1:
Question: I'm finding it hard to cope with sudden changes and uncertainty in my life. What practices can help me build resilience? Answer: there are a few different types of coping strategies that you can use to build resilient coping skills. the first type is the self-care approach. this is when you allow your partner to spend time with you instead of with other people. it is a way to let go of the worries and worries that are bothering you and instead focus on the positive aspects of your life and how you are able to connect with others. self care is an important part of many of my therapy work, so i encourage you to check out www.selfcaretherapy.com to see if they have one of these strategies. if not, i highly recommend seeing a therapist who specializes in working with people with anxiety.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I feel isolated and disconnected from those around me. What can I do to create a stronger support network? Answer: the best way to build a strong network of friends and family is to have a conversation with your counselor about what you are looking for in a counselor. this is a great opportunity to get to know one another and gain a deeper understanding of what is going on in your life. the more you have this conversation, the easier it will be for you to find someone to talk to.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I have trouble sleeping due to anxiety and racing thoughts. What are some effective methods to improve my sleep quality? Answer: sleep is one of the most important parts of a healthy and productive life. it is a fundamental part of our daily life and one that we all need to take very seriously. i would encourage you to check out www.suicidetherapy.com and www:therapistsupply.co.uk to find out more about what they have to offer.

Generated Output 1:
Question: I occasionally experience panic attacks that leave me feeling out of control. What strategies might help me prevent or manage these episodes? Answer: the best thing you can do for yourself and your panic attack symptoms is to talk to your primary care physician about the type of treatment you are considering. if you have not already done so, i highly recommend that you check out their website or call 1-800-273-8255 and speak with a local mental health professional to determine if there is a specific tr

In [ ]:
test_data_set = pd.read_csv("/content/counselchat-data_processed.csv")
test_data_set.head()

,questionTitle,questionText,topics,answerText,clean_questionTitle,clean_questionText,clean_answerText
0,Escalating disagreements between mother and wife,My wife and mother are having tense disagreeme...,Family Conflict,<p>What you are describing is something psycho...,escalating disagreements between mother and wife,my wife and mother are having tense disagreeme...,what you are describing is something psycholog...
1,I'm addicted to smoking. How can I stop?,"I'm planning to have baby, so I have to quit s...","Substance Abuse,Addiction",<p>Hi. Good for you in planning ahead to do wh...,i'm addicted to smoking. how can i stop?,"i'm planning to have baby, so i have to quit s...",hi. good for you in planning ahead to do what'...
2,Keeping secrets from my family,"I have secrets in my mind, and I don't know wh...",Family Conflict,<p>It sounds like keeping the secrets has beco...,keeping secrets from my family,"i have secrets in my mind, and i don't know wh...",it sounds like keeping the secrets has become ...
3,The Underlying Causes of Being Possessive,I am extremely possessive in my relationships ...,"Behavioral Change,Social Relationships",<p>Hi there. It's great you are able to realiz...,the underlying causes of being possessive,i am extremely possessive in my relationships ...,hi there. it's great you are able to realize t...
4,Can I control anxiety without medication?,I had a head injury a few years ago and my min...,Anxiety,<p>You didn't say what or how many medications...,can i control anxiety without medication?,i had a head injury a few years ago and my min...,you didn't say what or how many medications yo...


In [ ]:
col = ['questionTitle','questionText','topics','answerText']
test_data_set = test_data_set.drop(columns=col)
test_data_set = test_data_set.dropna()
test_data_set.head()

,clean_questionTitle,clean_questionText,clean_answerText
0,escalating disagreements between mother and wife,my wife and mother are having tense disagreeme...,what you are describing is something psycholog...
1,i'm addicted to smoking. how can i stop?,"i'm planning to have baby, so i have to quit s...",hi. good for you in planning ahead to do what'...
2,keeping secrets from my family,"i have secrets in my mind, and i don't know wh...",it sounds like keeping the secrets has become ...
3,the underlying causes of being possessive,i am extremely possessive in my relationships ...,hi there. it's great you are able to realize t...
4,can i control anxiety without medication?,i had a head injury a few years ago and my min...,you didn't say what or how many medications yo...


In [ ]:
test_data_set.shape

(1379, 3)

In [ ]:
test_df = test_data_set.sample(n=int(test_data_set.shape[0] * 0.2))
test_df.shape

(275, 3)

In [ ]:
test_df = test_df.drop(columns=['clean_questionTitle'])
test_df.head()

,clean_questionText,clean_answerText
528,me and the father of my child have been dating...,"well, as far as your boyfriend, sounds like he..."
1140,we've been in a long distance relationship for...,just ask him. i'm not sure how you saw his pho...
1298,my wife and i got separated because i asked ab...,"when ""threats"" of leaving, such as what you de..."
521,i feel so irritated and frustrated because of ...,yeah i get the same problem... look i'm not a ...
644,i am a survivor of domestic violence from a pa...,"hello, and thank you for your question. first ..."


In [15]:
!pip install nltk
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1297d7b05aaf527bb6ad9be48d195cb7bc936a2232ae1f35e9a44870d8babb9a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [16]:
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

In [17]:
model_path = "./fine_tuned_gpt2_in_mental_health_question"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [18]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
test_df['model-answer'] = ""

In [20]:
test_data.head()

,Unnamed: 0,clean_answerText,clean_questionText
1,1,"hello, and thank you for your question and see...",i'm going through some things with my feelings...
8,8,it sounds like you may be putting yourself las...,i'm going through some things with my feelings...
16,16,it sounds like you are having some pretty seve...,i'm going through some things with my feelings...
17,17,"you can, for sure... and the first step is to ...",i'm going through some things with my feelings...
19,19,feelings of worthlessness often originate from...,i'm going through some things with my feelings...


In [22]:
for idx , row in test_data.iterrows():
  prompt = "Question: " + row['clean_questionText'] + " Answer:"
  input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
  attention_mask = torch.ones(input_ids.shape, device=device)

  output = model.generate(
      input_ids,
      attention_mask = attention_mask,
      max_length=1024,
      num_beams=5,
      no_repeat_ngram_size=2,
      temperature=0.7,
      top_p=0.9,
      eos_token_id=tokenizer.eos_token_id,
  )
  generate_text = tokenizer.decode(output[0], skip_special_tokens=True)
  test_data.at[idx, 'model-answer'] = generate_text

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/p

In [35]:
test_data["extracted_answer"] = test_data["model-answer"].str.extract(r'Answer:\s*(.*)', expand=True)

In [38]:
test_data=test_data.drop(columns=['model-answer'])
test_data.head()

,Unnamed: 0,clean_answerText,clean_questionText,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,extracted_answer
1,1,"hello, and thank you for your question and see...",i'm going through some things with my feelings...,0.027040,0.484922,0.077388,0.154403,0.300332,it sounds like you are feeling pretty overwhel...
8,8,it sounds like you may be putting yourself las...,i'm going through some things with my feelings...,0.039812,0.324607,0.105079,0.153578,0.345311,it sounds like you are feeling pretty overwhel...
16,16,it sounds like you are having some pretty seve...,i'm going through some things with my feelings...,0.036302,0.434783,0.081575,0.168303,0.361889,it sounds like you are feeling pretty overwhel...
17,17,"you can, for sure... and the first step is to ...",i'm going through some things with my feelings...,0.025417,0.352564,0.083601,0.150641,0.332694,it sounds like you are feeling pretty overwhel...
19,19,feelings of worthlessness often originate from...,i'm going through some things with my feelings...,0.002037,0.095427,0.019960,0.055666,0.177917,it sounds like you are feeling pretty overwhel...


In [23]:
smoothie = SmoothingFunction().method1
rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougel"], use_stemmer=True)

In [ ]:
!pip install ace-tools

In [39]:
import nltk

# Download missing NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource

rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

bleu_scores, rouge_1_scores, rouge_2_scores, rouge_l_scores, meteor_scores = [], [], [], [], []

for idx, rows in test_data.iterrows():
    reference = rows['clean_answerText']
    hypothesis = rows['extracted_answer']

    # BLEU Score
    bleu_score = sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)
    bleu_scores.append(bleu_score)

    # ROUGE Score
    scores = rouge.score(reference, hypothesis)
    rouge_1_scores.append(scores["rouge1"].fmeasure)
    rouge_2_scores.append(scores["rouge2"].fmeasure)
    rouge_l_scores.append(scores["rougeL"].fmeasure)

    # METEOR Score - Fix: Tokenize both reference and hypothesis
    tokenized_reference = nltk.word_tokenize(reference)
    tokenized_hypothesis = nltk.word_tokenize(hypothesis)

    meteor_score_value = meteor_score([tokenized_reference], tokenized_hypothesis)
    meteor_scores.append(meteor_score_value)

# Store scores in DataFrame
test_data['BLEU'] = bleu_scores
test_data['ROUGE-1'] = rouge_1_scores
test_data['ROUGE-2'] = rouge_2_scores
test_data['ROUGE-L'] = rouge_l_scores
test_data['METEOR'] = meteor_scores

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [40]:
print(f"Average BLEU Score: {test_data['BLEU'].mean()}")
print(f"Average ROUGE-1 Score: {test_data['ROUGE-1'].mean()}")
print(f"Average ROUGE-2 Score: {test_data['ROUGE-2'].mean()}")
print(f"Average ROUGE-L Score: {test_data['ROUGE-L'].mean()}")
print(f"Average METEOR Score: {test_data['METEOR'].mean()}")

Average BLEU Score: 0.011768038208310507
Average ROUGE-1 Score: 0.31782152892584087
Average ROUGE-2 Score: 0.05167372249050384
Average ROUGE-L Score: 0.14870927392174188
Average METEOR Score: 0.20626782200325622
